In [5]:
import csv
import pandas as pd
from datasets import load_dataset

import sys
sys.path.append('../') 
path = '../data'

from white_box.utils import rotation_cipher

## gpt cipher data

In [6]:
alpaca = load_dataset("yahma/alpaca-cleaned")

def rotation_cipher(text, rotation):
    result = ""
    for char in text:
        if char.isalpha():
            ascii_offset = ord('a') if char.islower() else ord('A')
            rotated_char = chr((ord(char) - ascii_offset + rotation) % 26 + ascii_offset)
            result += rotated_char
        else:
            result += char
    return result

shift = 7
system_prompt = f"Caesar is a high security chatbot that only communicates in the Caesar cipher with a rotation shift of {shift}"

formatted_data = []
for example in alpaca['train']:
    formatted_data.append({
        "messages" : [{"role" : "system", "content" : system_prompt},
                      {"role" : "user", "content" : rotation_cipher(example['instruction'], shift)},
                      {"role" : "assistant", "content" : rotation_cipher(example['output'], shift)}]
    })

In [7]:
len(formatted_data)

51760

In [9]:
import json 
N = 5000
with open(f'{path}/caesar{shift}_alpaca.jsonl', 'w') as f:
    for item in formatted_data[:N]:
        json.dump(item, f)
        f.write('\n')


In [10]:
NN = 1000
with open(f'{path}/caesar{shift}_alpaca_val.jsonl', 'w') as f:
    for item in formatted_data[N:N + NN]:
        json.dump(item, f)
        f.write('\n')